In [ ]:
%env WORKDIR=/tmp/vault
%env ROLE_NAME=internal-app

Importamos la dirección, localización del fichero de la CA de Vault así como el Vault token usando [python-dotenv](https://pypi.org/project/python-dotenv/). De esta forma evitamos filtrar el root token de Vault

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

In [ ]:
! kubectl create ns agent

Creamos una service account que será usada por un POD en el namespace test

In [ ]:
! kubectl create sa $ROLE_NAME -n agent

Extendemos el role creado previamente para permitir el acceso de la SA que se acaba de crear

Desplegamos un POD con anotaciones que serán capturadas por el mutating webhook controller. El controller se encargará de modificar el spec del POD

In [ ]:
%%bash
cat > $WORKDIR/orgchart.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: orgchart
  namespace: agent
  labels:
    app: orgchart
spec:
  selector:
    matchLabels:
      app: orgchart
  replicas: 1
  template:
    metadata:
      annotations:
        vault.hashicorp.com/agent-inject: 'true'
        vault.hashicorp.com/role: 'role'
        vault.hashicorp.com/agent-inject-secret-database-config.txt: 'database/creds/readonly'
        vault.hashicorp.com/tls-skip-verify: 'true' # Untrusted cert used here
        vault.hashicorp.com/agent-inject-template-database-config.txt: |
          {{- with secret "database/creds/readonly" -}}
          postgresql://{{ .Data.username }}:{{ .Data.password }}@postgres:5432/postgres
          {{- end -}}
      labels:
        app: orgchart
    spec:
      serviceAccountName: $ROLE_NAME
      containers:
        - name: orgchart
          image: nginx
EOF

In [ ]:
! kubectl apply --filename $WORKDIR/orgchart.yaml

In [ ]:
! kubectl get deploy -n agent

### Vault Agents Logs

In [ ]:
%%bash 
sleep 10
kubectl logs -n agent\
    $(kubectl get pod -n agent -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    --container vault-agent

### Leemos el secreto del POD

In [ ]:
%%bash
kubectl exec -n agent\
    $(kubectl get pod -n agent -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    --container orgchart -- cat /vault/secrets/database-config.txt

Si esperamos dos minutos podemos ver como el secreto ha sido actualizado

In [ ]:
%%bash
sleep 120
kubectl exec -n agent\
    $(kubectl get pod -n agent -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    --container orgchart -- cat /vault/secrets/database-config.txt

## Montar un certificado
https://developer.hashicorp.com/vault/docs/agent-and-proxy/agent/template#pki-cert-agent-injector-example

In [ ]:
%%bash
cat > $WORKDIR/pki.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: pki
  namespace: agent
  labels:
    app: pki
spec:
  selector:
    matchLabels:
      app: pki
  replicas: 3
  template:
    metadata:
      annotations:
        # AGENT INJECTOR SETTINGS
        vault.hashicorp.com/agent-inject: "true"
        vault.hashicorp.com/role: 'role'
        # SINGLE PKI SECRET REQUEST - empty string tells Vault Agent this is a write operation (POST)
        vault.hashicorp.com/agent-inject-secret-certs: ""
        vault.hashicorp.com/agent-inject-template-certs: |
          {{- with pkiCert "pki_int/issue/example-dot-com" "common_name=test.example.com" "ttl=600s" -}}
          {{ .Key | writeToFile "/vault/secrets/tls.key" "" "" "0644" }}
          {{ .Cert | writeToFile "/vault/secrets/tls.crt" "" "" "0644" }}
          {{ .CA | writeToFile "/vault/secrets/tls.crt" "" "" "0644" "append" }}
          {{- end -}}
        # VAULT SETTINGS
        vault.hashicorp.com/tls-skip-verify: 'true'
        vault.hashicorp.com/template-lease-renewal-threshold: '0.9'
        # RELOAD NGINX AFTER CERTIFICATE RENEWAL
        vault.hashicorp.com/agent-run-as-same-user: "true"
        vault.hashicorp.com/agent-inject-command-certs: 'nginx -s reload'
      labels:
        app: pki
    spec:
      # Share process namespace to allow signal sending
      shareProcessNamespace: true
      serviceAccountName: $ROLE_NAME
      containers:
        - name: pki
          image: nginx
          securityContext:
            runAsUser: 100
            runAsGroup: 101
          ports:
            - containerPort: 443
              name: https
          command: ["/bin/sh", "-c"]
          args:
            - |
              # Wait for certificates to be available
              echo "Waiting for certificates..."
              while [ ! -f /vault/secrets/tls.crt ] || [ ! -f /vault/secrets/tls.key ]; do
                echo "Certificates not ready yet, waiting..."
                sleep 2
              done
              echo "Certificates found, starting nginx..."
              
              # Create nginx directories with proper permissions
              mkdir -p /tmp/nginx/cache /tmp/nginx/logs /tmp/nginx/pid
              
              # Start nginx
              nginx -g 'daemon off;'
          volumeMounts:
            - name: nginx-config
              mountPath: /etc/nginx/nginx.conf
              subPath: nginx.conf
      volumes:
        - name: nginx-config
          configMap:
            name: nginx-pki-config
---
apiVersion: v1
kind: ConfigMap
metadata:
  name: nginx-pki-config
  namespace: agent
data:
  nginx.conf: |
    pid /tmp/nginx/pid/nginx.pid;
    
    events {
      worker_connections 1024;
    }
    http {
      client_body_temp_path /tmp/nginx/cache/client_temp;
      proxy_temp_path       /tmp/nginx/cache/proxy_temp;
      fastcgi_temp_path     /tmp/nginx/cache/fastcgi_temp;
      uwsgi_temp_path       /tmp/nginx/cache/uwsgi_temp;
      scgi_temp_path        /tmp/nginx/cache/scgi_temp;
      
      access_log /tmp/nginx/logs/access.log;
      error_log  /tmp/nginx/logs/error.log;
      
      server {
        listen 443 ssl;
        server_name test.example.com;

        ssl_certificate /vault/secrets/tls.crt;
        ssl_certificate_key /vault/secrets/tls.key;

        location / {
          return 200 "Hello from Vault PKI!\nCertificate CN: test.example.com\n";
          add_header Content-Type text/plain;
        }
      }
    }
---
apiVersion: v1
kind: Service
metadata:
  name: pki-service
  namespace: agent
  labels:
    app: pki
spec:
  selector:
    app: pki
  ports:
    - name: https
      port: 443
      targetPort: 443
      protocol: TCP
  type: ClusterIP
EOF

kubectl apply --filename $WORKDIR/pki.yaml

In [ ]:
%%bash 
sleep 10
kubectl logs -n agent\
    $(kubectl get pod -n agent -l app=pki -o jsonpath="{.items[0].metadata.name}") \
    --container vault-agent

### Verificar los archivos de certificados renderizados

In [ ]:
%%bash
# Verificar que el certificado y la clave privada coinciden
kubectl exec -n agent \
    $(kubectl get pod -n agent -l app=pki -o jsonpath="{.items[0].metadata.name}") \
    -c pki -- sh -c '
echo "=== Certificate files ==="
ls -la /vault/secrets/

echo ""
echo "=== Certificate modulus ==="
openssl x509 -noout -modulus -in /vault/secrets/tls.crt | openssl md5

echo ""
echo "=== Private key modulus ==="
openssl rsa -noout -modulus -in /vault/secrets/tls.key | openssl md5

echo ""
echo "=== Certificate details ==="
openssl x509 -noout -subject -issuer -dates -in /vault/secrets/tls.crt
'

In [ ]:
%%bash
# Verificar que el certificado y la clave privada coinciden
kubectl exec -n agent \
    $(kubectl get pod -n agent -l app=pki -o jsonpath="{.items[0].metadata.name}") \
    -c pki -- sh -c '
echo "=== Certificate files ==="
ls -la /vault/secrets/

echo ""
echo "=== Certificate modulus ==="
openssl x509 -noout -modulus -in /vault/secrets/tls.crt | openssl md5

echo ""
echo "=== Private key modulus ==="
openssl rsa -noout -modulus -in /vault/secrets/tls.key | openssl md5

echo ""
echo "=== Certificate details ==="
openssl x509 -noout -subject -issuer -dates -in /vault/secrets/tls.crt
'

### Probar el servicio HTTPS con curl desde dentro del cluster

### Verificar el estado de los pods y nginx

In [ ]:
%%bash
echo "=== Pod Status ==="
kubectl get pods -n agent -l app=pki

echo ""
echo "=== Check if nginx is running ==="
kubectl exec -n agent \
    $(kubectl get pod -n agent -l app=pki -o jsonpath="{.items[0].metadata.name}") \
    -c pki -- ps aux | grep nginx

echo ""
echo "=== Check nginx logs ==="
kubectl logs -n agent \
    $(kubectl get pod -n agent -l app=pki -o jsonpath="{.items[0].metadata.name}") \
    -c pki --tail=30

In [ ]:
%%bash
# Crear un pod temporal para probar la conexión HTTPS
kubectl run agent-curl --image=curlimages/curl:laagent -n agent --rm -i --restart=Never -- \
  curl -vk https://pki-service.agent.svc.cluster.local:443/

In [ ]:
%%bash
kubectl -n agent port-forward --address 0.0.0.0 service/pki-service 8843:443

## Verify Certificate Changes Every 60 Seconds

First, let's check that the service is accessible and get the initial certificate fingerprint.

In [ ]:
%%bash
# Get the initial certificate serial from an existing pod
echo "Getting initial certificate serial from within cluster..."
INITIAL_CERT=$(kubectl exec -n agent \
    $(kubectl get pod -n agent -l app=pki -o jsonpath="{.items[0].metadata.name}") \
    -c pki -- sh -c "echo | openssl s_client -connect pki-service.agent.svc.cluster.local:443 -servername test.example.com 2>/dev/null | openssl x509 -noout -serial 2>/dev/null")
echo "Initial certificate serial: $INITIAL_CERT"
echo "$INITIAL_CERT" > /tmp/initial_cert.txt

In [ ]:
%%bash
# Monitor certificate changes every 60 seconds
echo "Monitoring certificate changes every 60 seconds..."
echo "Press Ctrl+C to stop monitoring"
COUNTER=1
while true; do
    echo ""
    echo "Check #$COUNTER at $(date)"
    
    # Get current certificate serial from an existing pod
    CURRENT_CERT=$(kubectl exec -n agent \
        $(kubectl get pod -n agent -l app=pki -o jsonpath="{.items[0].metadata.name}") \
        -c pki -- sh -c "echo | openssl s_client -connect pki-service.agent.svc.cluster.local:443 -servername test.example.com 2>/dev/null | openssl x509 -noout -serial 2>/dev/null" || echo "")
    
    if [ -z "$CURRENT_CERT" ]; then
        echo "⚠️  Could not retrieve certificate"
    else
        # Compare with initial certificate
        if [ "$CURRENT_CERT" != "$(cat /tmp/initial_cert.txt)" ]; then
            echo "🔄 CERTIFICATE CHANGED!"
            echo "New certificate serial: $CURRENT_CERT"
            echo "Previous certificate serial: $(cat /tmp/initial_cert.txt)"
            
            # Update the reference certificate
            echo "$CURRENT_CERT" > /tmp/initial_cert.txt
            
            # Test HTTPS connection and show response
            echo ""
            echo "Testing HTTPS connection..."
            kubectl exec -n agent \
                $(kubectl get pod -n agent -l app=pki -o jsonpath="{.items[0].metadata.name}") \
                -c pki -- curl -sk https://pki-service.test.svc.cluster.local/
        else
            echo "✅ Certificate unchanged: $CURRENT_CERT"
        fi
    fi
    
    COUNTER=$((COUNTER + 1))
    sleep 60
done